# Video Similarity Search

This notebook loads the existing Pixeltable pipeline from `d2.ipynb`, inserts new videos, and runs a similarity search over transcribed sentences.

In [1]:
import pixeltable as pxt
import pixeltable.functions as pxtf

In [2]:
t = pxt.get_table('olympics.skates')
ac_view = pxt.get_table('olympics.audio_chunks')
sent_view = pxt.get_table('olympics.transcript_chunks')
t

/Users/alison-pxt/Documents/Github/pxt-d2/.venv/lib/python3.13/site-packages/pixeltable/env.py:494: UserWarning: Progress reporting is disabled because ipywidgets is not installed. To fix this, run: `pip install ipywidgets`
  warnings.warn(


Connected to Pixeltable database at: postgresql+psycopg://postgres:@/pixeltable?host=/Users/alison-pxt/.pixeltable/pgdata


Loading weights: 100%|██████████| 199/199 [00:00<00:00, 1919.35it/s, Materializing param=pooler.dense.weight]                        
MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


table 'olympics/skates'

 Column Name   Type                       Computed With Comment
       video  Video                                            
       audio  Audio  extract_audio(video, format='wav')

In [3]:
t.insert([
    {'video': 'media/olympics-glenn.mp4'},
    {'video': 'media/olympics-liu.mp4'}
])

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 2133.66it/s, Materializing param=pooler.dense.weight]                        
MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Inserted 221 rows with 0 errors in 16.23 s (13.62 rows/s)


221 rows inserted.

In [4]:
sent_view.count()

205

In [7]:
query_text = 'gold medal'
sim = sent_view.text.similarity(string=query_text)

results = (
    sent_view
    .where(sim >= 0.3)
    .order_by(sim, asc=False)
    .select(sent_view.text, sent_view.pos, sent_view.audio_chunk, similarity=sim)
    .limit(5)
    .collect()
)
results

text,pos,audio_chunk,similarity
medals.,0,,0.708
"The first medal for the American women since 2006, when Sasha Cohen won her Olympic medal.",0,,0.621
Probably means a medal.,1,,0.619
and win gold.,0,,0.495
"She said, I don't need a medal.",0,,0.437
